In [1]:
import numpy as np

In [2]:
m = 3
m2 = m ** 2
q = np.zeros(m2)
q[m2 // 2] = 1

In [3]:
def get_P(m, p_up, p_down, p_left, p_right):
    m2 = m ** 2
    P = np.zeros((m2, m2))
    ix_map = {i + 1: (i //m, i % m) for i in range(m2)}
    for i in range(m2):
        for j in range(m2):
            r1, c1 = ix_map[i+1]
            r2, c2 = ix_map[j+1]
            rdiff = r1 - r2
            cdiff = c1 - c2
            if rdiff == 0:
                if cdiff == 1:
                    P[i, j] = p_left
                elif cdiff == -1:
                    P[i, j] = p_right
                elif cdiff == 0:
                    if r1 == 0:
                        P[i,j] += p_down
                    elif r1 == m - 1:
                        P[i, j] += p_up
                    if c1 == 0: 
                        P[i, j] += p_left
                    elif c1 == m - 1:
                        P[i, j] += p_right
            elif rdiff == 1:
                if cdiff == 0:
                    P[i,j] = p_down
            elif rdiff == -1:
                if cdiff == 0:
                    P[i, j] = p_up
    return P

In [4]:
P = get_P(3, 0.2, 0.3, 0.25, 0.25)

In [5]:
n = 1
Pn = np.linalg.matrix_power(P, n)
np.matmul(q, Pn)

array([0.  , 0.3 , 0.  , 0.25, 0.  , 0.25, 0.  , 0.2 , 0.  ])

### Sample path in an ergodic Markov Chain

In [6]:
from scipy.stats import itemfreq

In [7]:
s=4
n = 10**6
visited = [s]
for t in range(n):
    s = np.random.choice(m2, p=P[s, :])
    visited.append(s)
    
itemfreq(visited)

<ipython-input-7-13dd9aedb1bc>:8: DeprecationWarning: `itemfreq` is deprecated!
`itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`
  itemfreq(visited)


array([[     0, 157498],
       [     1, 156766],
       [     2, 158538],
       [     3, 105297],
       [     4, 105310],
       [     5, 106238],
       [     6,  69636],
       [     7,  70078],
       [     8,  70640]])

In [8]:
P = np.zeros((m2 + 1, m2 + 1))
P[:m2, :m2] = get_P(3, 0.2, 0.3, 0.25, 0.25)
for i in range(m2):
    P[i, m2] = P[i, i]
    P[i, i] = 0
P[m2, m2] = 1

In [9]:
n = 10 ** 5
avg_rewards = np.zeros(m2)
for s in range(9):
    for i in range(n):
        crashed = False
        s_next = s
        episode_reward = 0
        while not crashed:
            s_next = np.random.choice(m2 + 1, p=P[s_next, :])
            if s_next < m2:
                episode_reward += 1
            else:
                crashed = True
        avg_rewards[s] += episode_reward
        
avg_rewards /= n

In [10]:
def estimate_state_values(P, m2, threshold):
    v = np.zeros(m2 + 1)
    max_change = threshold
    terminal_state = m2
    while max_change >= threshold:
        max_change = 0
        for s in range(m2 + 1):
            v_new = 0
            for s_next in range(m2 + 1):
                r = 1 * (s_next != terminal_state)
                v_new += P[s, s_next] * (r + v[s_next])
            max_change = max(max_change, np.abs(v[s] - v_new))
            v[s] = v_new
    return np.round(v,2)

In [11]:
estimate_state_values(P, m2, 0.01)

array([1.46, 2.11, 1.47, 2.44, 3.41, 2.44, 1.98, 2.82, 1.99, 0.  ])